# Run simulations

Runs a simulation given base params, adds noise then fits multiple models to the same simulation.
Saves the results as a dictionary including keys:
* ```metadata``` - a dictionary with information about the simulation
* ```noisy_fits``` - a dictionary whose keys are integers indicating the model number. <br> Each model number contains the fit results of that model including ```ydata_tofit``` the data to be fit,  ```phi_init_base``` as the initial condition to the fit and ```phi1``` as the resulting best fit parameter vector.

Assumes a global variable to exist:
* ```CURDATAPOINT``` is an integer that is going to index simulation number
* ```save_str``` is as string to add to the file name in order to avoid overriding, by default it can be an empty string

In [30]:
# Uncomment the lines below to save via local variables
# CURDATAPOINT = 1;
# save_str = ""

In [ ]:
# Running multiple simulations with the same settings below (do this from console, DO NOT UNCOMMENT THESE):
# CURDATAPOINT = 0
# save_str = "test_fits_"
# for j11 = 1:100
#       CURDATAPOINT = j11
#       nbinclude("run_Simulations.ipynb")
#       println(CURDATAPOINT) # Shows progress
# end

In [31]:
include("../BK_functions/bk_setup_script.jl"); 

In [12]:
# Load the parameters from Fig 2 example:
using JLD

phi_base = phi0; # Use the base parameters as true parameters to be estimated via fits through noise

# Alternatively, use diverging parameters as base ones to be estimated
# phi_base = load("results/noisy_fits_8.jld", "noisy_fits")[10]["phi1_orig"]
# phi_base = load("results/noisy_fits_5.jld", "noisy_fits")[10]["phi1_orig"]

phi_base = load("results/noisy_fits_7.jld", "noisy_fits")[10]["phi1_orig"]

8-element Array{Float64,1}:
 0.00018292
 0.781112  
 0.128178  
 0.763019  
 6.01064e-5
 6.19077   
 4.5954    
 3.50126   

In [35]:
metadata = Dict("data_noise"=>"10%", "num_inits"=>24, "init_param_noise"=>"90%", "phi_base"=>phi_base, "phi1_orig"=>phi_base) # phi1_orig key added for historical compatibility 

Dict{ASCIIString,Any} with 5 entries:
  "phi_base"         => [2.2e-6,0.42,0.1026,0.58,3.9e-5,6.16,30.4,2.0]
  "data_noise"       => "10%"
  "phi1_orig"        => [2.2e-6,0.42,0.1026,0.58,3.9e-5,6.16,30.4,2.0]
  "init_param_noise" => "90%"
  "num_inits"        => 24

In [36]:
# Simulate appropriate noisy data from the phi_base parameters via the full model

ydata_noiseless = BK_simulator(phi_base, hcat(x_grid...), model_id=0);
ydata_orig = ydata_noiseless.*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative)
ydata_log = log10(ydata_noiseless).*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative, but in log space assay)
;

# Fit models and initial conditions to the simulated data

In [37]:
# Choose models, initial conditions and data to fit

model_nums = [52];
init_conds = Array(Any,length(model_nums))
data_sets = Array(Any,length(model_nums))

# For appropriate initial conditions for reduced models, load MBAM results
using JLD
outp = load("results/MBAM_outp_orig.jld", "outp");
outp_log10 = load("results/MBAM_outp_log10.jld", "outp_log10");

n = 0;
for model in model_nums
    n += 1
    if model <=16
        init_conds[n] = exp(outp[1][model-9,1]); # Appropriate parameter vector for the model
        data_sets[n] = ydata_orig;
    elseif model > 50
        init_conds[n] = exp(outp_log10[1][model-50,1]); # Appropriate parameter vector for the model
        data_sets[n] = ydata_log;
    end
end

In [38]:
# Generate the "noisy_fits" dictionary where we save the fit results to
noisy_fits = Dict();

In [ ]:
# Iterate over all models to be fit and do the fit
using LsqFit
n = 0;
for model in model_nums
    n+=1;
    ydata_tofit = data_sets[n];
    phi_init_base = init_conds[n];

    # Store multiple fits (different initial point for LM, takes around 5 sec / fit)
    all_fits = Array(Any, 24)
    all_costs = Array(Any, length(all_fits))

    for i1 = 1:length(all_fits)
        if i1==1
            phi_init = phi_init_base; # Start the first fit from initial params
        else
            phi_init = phi_init_base.*(1+(2*rand(size(phi_init_base))-1)*0.9) # Add up to 90% noise to initial params
        end
        all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model), hcat(x_grid...), ydata_tofit, log(phi_init));
        all_costs[i1] = sum(all_fits[i1].resid.^2);
    end

    # Find the best fit
    phi1 = exp(all_fits[findmin(all_costs)[2]].param);

    # Saving fit results:
    res = Dict()
    res["ydata_tofit"] = ydata_tofit
    res["phi_init_base"] = phi_init_base
    res["phi1"] = phi1
    # Added for historical compatibility
    res["ydata_orig"] = ydata_tofit
    res["phi0_orig"] = phi_init_base
    res["phi1_orig"] = phi1
    res["ydata_log"] = ydata_tofit
    res["phi0_log"] = phi_init_base
    res["phi1_log"] = phi1
    
    
    noisy_fits[model] = res
end

In [ ]:
save("results/noisy_fits_"*save_str*string(CURDATAPOINT)*".jld", "noisy_fits", noisy_fits, "metadata", metadata);